# Clustering Crypto

In [6]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from urllib.request import Request, urlopen
import json
from path import Path
import plotly.express as px

### Fetching Cryptocurrency Data

In [7]:
# Use the following endpoint to fetch json data
url = "https://min-api.cryptocompare.com/data/all/coinlist"
json_request = Request(url)
json_response = urlopen(json_request)
data = json_response.read()
json_data = json.loads(data)

In [8]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame.
api_crypto_df = pd.DataFrame(json_data['Data']).T
api_crypto_df.head()


,Id,Url,ImageUrl,ContentCreatedOn,Name,Symbol,CoinName,FullName,Description,AssetTokenStatus,...,AssetLaunchDate,MaxSupply,MktCapPenalty,PlatformType,BuiltOn,SmartContractAddress,DecimalPoints,Difficulty,IsUsedInDefi,IsUsedInNft
42,4321,/coins/42/overview,/media/35650717/42.jpg,1427211129,42,42,42 Coin,42 Coin (42),Everything about 42 coin is 42 - apart from th...,N/A,...,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
300,749869,/coins/300/overview,/media/27010595/300.png,1517935016,300,300,300 token,300 token (300),300 token is an ERC20 token. This Token was cr...,N/A,...,2017-07-01,300,0,token,ETH,0xaec98a708810414878c3bcdf46aad31ded4a4557,18,NaN,NaN,NaN
365,33639,/coins/365/overview,/media/352070/365.png,1480032918,365,365,365Coin,365Coin (365),365Coin is a Proof of Work and Proof of Stake ...,N/A,...,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
404,21227,/coins/404/overview,/media/35650851/404-300x300.jpg,1466100361,404,404,404Coin,404Coin (404),404 is a PoW/PoS hybrid cryptocurrency that al...,N/A,...,0000-00-00,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
433,926547,/coins/433/overview,/media/34836095/433.png,1541597321,433,433,433 Token,433 Token (433),433 Token is a decentralised soccer platform t...,Finished,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Alternatively, use the provided csv file:
file_path = Path("/Users/Jimmy/Dropbox/rice-hou-fin-pt-12-2020-u-c copy/2_Homeworks/13-AWS-Lex/Instructions/Starter_Files/Resources/crypto_data.csv")
df_crypto = pd.read_csv(file_path)
df_crypto.head()
# file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
df_crypto = pd.DataFrame(df_crypto) 
df_crypto.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


### Data Preprocessing

In [12]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
df =df_crypto[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
df.head()

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [13]:
# Keep only cryptocurrencies that are trading
crypto_df = df.loc[df['IsTrading'] == True]

In [14]:
# Keep only cryptocurrencies with a working algorithm
crypto_df = crypto_df.loc[df_crypto['Algorithm'] != 'N/A']

In [15]:
# Remove the "IsTrading" column
crypto_df = crypto_df.drop(columns='IsTrading')

In [16]:
# Remove rows with at least 1 null value
crypto_df= crypto_df.dropna()

In [17]:
# Remove rows with cryptocurrencies having no coins mined
crypto_df = crypto_df.loc[crypto_df['TotalCoinsMined'] != 0]

In [18]:
# Drop rows where there are 'N/A' text values
crypto_df = crypto_df.loc[crypto_df['TotalCoinSupply'] != 'N/A']
crypto_df = crypto_df.loc[crypto_df['ProofType'] != 'N/A']


In [20]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df
names = crypto_df
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,Ethereum,Ethash,PoW,1.076842e+08,0


In [21]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
crypto_df = crypto_df.drop(columns='CoinName')



In [23]:
# Create dummy variables for text features
crypto_encoded = pd.get_dummies(data=crypto_df, columns=['Algorithm', 'ProofType' ])

In [24]:
# Standardize data
crypto_df_scaled = StandardScaler().fit_transform(crypto_encoded)

### Reducing Dimensions Using PCA

In [25]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(crypto_df_scaled)

In [26]:
# Create a DataFrame with the principal components data
pca_df = pd.DataFrame(
    data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index
)
pca_df.head()

,PC 1,PC 2,PC 3
0,-0.326659,0.978550,-0.622080
2,-0.309975,0.978662,-0.622456
5,2.310757,1.651394,-0.730346
7,-0.146255,-1.259505,0.196369
8,-0.159401,-1.985125,0.376481


### Clustering Crytocurrencies Using K-Means

#### Find the Best Value for `k` Using the Elbow Curve

In [27]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pca_df )
    inertia.append(km.inertia_)


# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [28]:
# Initialize the K-Means model
kmodel = KMeans(n_clusters=6, random_state=0)
# Fit the model
kmodel.fit(pca_df)
# Predict clusters
predictions = kmodel.predict(pca_df)
# Create a new DataFrame including predicted clusters and cryptocurrencies features
clustered_df = pd.DataFrame({
    "CoinName": names.CoinName,
    "Algorithm": crypto_df.Algorithm,
    "ProofType": crypto_df.ProofType,
    "TotalCoinsMined": crypto_df.TotalCoinsMined,
    "TotalCoinSupply": crypto_df.TotalCoinSupply,
    "PC 1": pca_df['PC 1'],
    "PC 2": pca_df['PC 2'],
    "PC 3": pca_df['PC 3'],
    "Class": kmodel.labels_,
    "Predictions": predictions, 
    },
)
clustered_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,Class,Predictions
0,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42,-0.326659,0.978550,-0.622080,2,2
2,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.309975,0.978662,-0.622456,2,2
5,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359,2.310757,1.651394,-0.730346,4,4
7,Bitcoin,SHA-256,PoW,1.792718e+07,21000000,-0.146255,-1.259505,0.196369,0,0
8,Ethereum,Ethash,PoW,1.076842e+08,0,-0.159401,-1.985125,0.376481,5,5


### Visualizing Results

#### 3D-Scatter with Clusters

In [29]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name='CoinName',
    hover_data= ['Algorithm'],
    color="Class",
    symbol="Class",
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

#### Table of Tradable Cryptocurrencies

In [36]:
# Table with tradable cryptos
columns = ['CoinName', 'Algorithm', 'ProofType', 'TotalCoinSupply', 'TotalCoinsMined', 'Class']
clustered_df.hvplot.table(columns)

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,Class]

In [41]:
# Print the total number of tradable cryptocurrencies
print(len(columns))

6


#### Scatter Plot with Tradable Cryptocurrencies

In [31]:
# Scale data to create the scatter plot
clustered_df['TotalCoinsMined'] = clustered_df['TotalCoinsMined'].astype(float) / 1000000000
clustered_df['TotalCoinSupply'] = clustered_df['TotalCoinSupply'].astype(float) / 1000000000

In [37]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
clustered_df.hvplot(
    kind="scatter", 
    x="TotalCoinsMined", 
    y="TotalCoinSupply", 
    c='Class', 
    colormap="viridis", 
    hover_cols=['CoinName']
)


:Scatter   [TotalCoinsMined]   (TotalCoinSupply,Class,CoinName)